In [1]:
import tensorflow as tf
import numpy as np
%load_ext autoreload
%autoreload 2

## Create the Resnet

In [2]:
from kerod.model.backbone.resnet import ResNet50PytorchStyle
test = np.zeros((1, 800, 800, 3))
model = ResNet50PytorchStyle(input_shape=[None, None, 3], weights=None)


## Download the tensorpack weights

In [3]:
!wget http://models.tensorpack.com/ResNet/ImageNet-ResNet50.npz

--2020-05-22 12:30:49--  http://models.tensorpack.com/ResNet/ImageNet-ResNet50.npz
Resolving models.tensorpack.com (models.tensorpack.com)... 185.207.105.29
Connecting to models.tensorpack.com (models.tensorpack.com)|185.207.105.29|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 95179469 (91M) [application/octet-stream]
Saving to: ‘ImageNet-ResNet50.npz.2’

ImageNet-ResNet50.n 100%[===================>]  90.77M  18.9MB/s    in 4.9s    

2020-05-22 12:30:54 (18.6 MB/s) - ‘ImageNet-ResNet50.npz.2’ saved [95179469/95179469]



## Load the weights

In [3]:
tensorpack_weights = {}
with np.load('ImageNet-ResNet50.npz') as data:
    for key in data.keys():
        tensorpack_weights[key] = data[key]

## Create the mapping dictionary

In [4]:
convert_to = {
    'conv1_conv/kernel:0': 'conv0/W:0',
    'conv1_bn/gamma:0': 'conv0/bn/gamma:0',
    'conv1_bn/beta:0': 'conv0/bn/beta:0',
    'conv1_bn/moving_mean:0': 'conv0/bn/mean/EMA:0',
    'conv1_bn/moving_variance:0': 'conv0/bn/variance/EMA:0',
    'moving_variance:0': 'variance/EMA:0',
    'moving_mean:0':'mean/EMA:0',
    'kernel:0':'W:0',
    'resnet50/': '', 
}

In [5]:
for var in model.variables:
    print(var.name)

conv1_conv/kernel:0
conv1_bn/gamma:0
conv1_bn/beta:0
conv1_bn/moving_mean:0
conv1_bn/moving_variance:0
resnet50/group0/block0/conv1/kernel:0
resnet50/group0/block0/conv1/bn/gamma:0
resnet50/group0/block0/conv1/bn/beta:0
resnet50/group0/block0/conv1/bn/moving_mean:0
resnet50/group0/block0/conv1/bn/moving_variance:0
resnet50/group0/block0/conv2/kernel:0
resnet50/group0/block0/conv2/bn/gamma:0
resnet50/group0/block0/conv2/bn/beta:0
resnet50/group0/block0/conv2/bn/moving_mean:0
resnet50/group0/block0/conv2/bn/moving_variance:0
resnet50/group0/block0/convshortcut/kernel:0
resnet50/group0/block0/conv3/kernel:0
resnet50/group0/block0/convshortcut/bn/gamma:0
resnet50/group0/block0/convshortcut/bn/beta:0
resnet50/group0/block0/convshortcut/bn/moving_mean:0
resnet50/group0/block0/convshortcut/bn/moving_variance:0
resnet50/group0/block0/conv3/bn/gamma:0
resnet50/group0/block0/conv3/bn/beta:0
resnet50/group0/block0/conv3/bn/moving_mean:0
resnet50/group0/block0/conv3/bn/moving_variance:0
resnet50/g

In [6]:
used = set()
for var in model.variables:
    name_var = var.name
    for key, value in convert_to.items():
        name_var = name_var.replace(key, value)
    if name_var in used:
        print('what')
    weights = tensorpack_weights[name_var]
    used.add(name_var)
    var.assign(weights)
    np.testing.assert_almost_equal(weights, var.numpy())


In [7]:
len(used)

265

In [24]:
model.save_weights('resnet50_tensorpack_converted.h5')

In [12]:
model.load_weights('resnet50_tensorpack_converted.h5')

In [30]:
!pip install Pillow
!wget https://images.lanouvellerepublique.fr/image/upload/t_1020w/f_auto/5b95be27be7744fb5c8b467b.jpg

  Using cached Pillow-7.1.2-cp37-cp37m-manylinux1_x86_64.whl (2.1 MB)
You should consider upgrading via the '/home/egarreau/.pyenv/versions/3.7.2/envs/tensorflow/bin/python -m pip install --upgrade pip' command.


In [8]:
from PIL import Image

img = Image.open('5b95be27be7744fb5c8b467b.jpg')
img = np.array(img)

In [9]:
from kerod.utils.training import freeze_batch_normalization
freeze_batch_normalization(model)


In [10]:
outputs = model(img[None])

In [37]:
outputs[2][0, 2]

<tf.Tensor: shape=(64, 1024), dtype=float32, numpy=
array([[0.        , 0.14657381, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.32357076, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.7661909 , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.27262533, 0.        , 0.        , ..., 0.        , 1.5332065 ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.0833144 ,
        0.        ],
       [0.        , 1.0755405 , 0.        , ..., 0.        , 0.        ,
        0.        ]], dtype=float32)>

In [12]:
for i, layer in enumerate(model.layers):
    print(i)
    print(layer.name)

0
input_1
1
preprocess_input
2
padd_for_aligning_pixels
3
conv1_conv
4
conv1_bn
5
conv1_relu
6
pool1_pad
7
pool1_pool
8
resnet50/group0/block0/conv1
9
resnet50/group0/block0/conv1/bn
10
resnet50/group0/block0/conv1/relu
11
resnet50/group0/block0/conv2
12
resnet50/group0/block0/conv2/bn
13
resnet50/group0/block0/conv2/relu
14
resnet50/group0/block0/convshortcut
15
resnet50/group0/block0/conv3
16
resnet50/group0/block0/convshortcut/bn
17
resnet50/group0/block0/conv3/bn
18
add
19
resnet50/group0/block0/last_relu
20
resnet50/group0/block1/conv1
21
resnet50/group0/block1/conv1/bn
22
resnet50/group0/block1/conv1/relu
23
resnet50/group0/block1/conv2
24
resnet50/group0/block1/conv2/bn
25
resnet50/group0/block1/conv2/relu
26
resnet50/group0/block1/conv3
27
resnet50/group0/block1/conv3/bn
28
add_1
29
resnet50/group0/block1/last_relu
30
resnet50/group0/block2/conv1
31
resnet50/group0/block2/conv1/bn
32
resnet50/group0/block2/conv1/relu
33
resnet50/group0/block2/conv2
34
resnet50/group0/block2/con

In [ ]:
target =  21 # resnet50/group0/block0/last_relu

In [61]:
target =  5
model.layers[target].output

<tf.Tensor 'conv1_relu/Identity:0' shape=(None, None, None, 64) dtype=float32>

In [62]:
tmp_model = tf.keras.Model(
    inputs=model.inputs,
    outputs=model.layers[target].output)
val = tmp_model(img[None])

In [64]:
#padding_tp = np.load('/home/egarreau/Documents/machine_learning/tensorpack/examples/FasterRCNN/padding.npz')
conv_tensorpack = np.load('/home/egarreau/Documents/machine_learning/tensorpack/examples/FasterRCNN/conv.npy')

In [67]:
np.testing.assert_array_almost_equal(conv_tensorpack, val)

AssertionError: 
Arrays are not almost equal to 6 decimals

Mismatched elements: 1871 / 9437184 (0.0198%)
Max absolute difference: 3.0994415e-06
Max relative difference: 0.2760105
 x: array([[[[ 3.397575,  0.875192,  1.134879, ...,  1.156682,  2.317088,
           1.015946],
         [ 4.162702,  2.006068,  0.961679, ...,  1.406666,  2.530519,...
 y: array([[[[3.397575, 0.875192, 1.134879, ..., 1.156682, 2.317088,
          1.015946],
         [4.162702, 2.006068, 0.961679, ..., 1.406666, 2.530519,...

In [51]:
val

<tf.Tensor: shape=(1, 289, 513, 64), dtype=float32, numpy=
array([[[[0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         ...,
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ]],

        [[0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [3.397575  , 0.87519157, 1.1348791 , ..., 1.1566823 ,
          2.317088  , 1.0159464 ],
         [4.1627016 , 2.0060682 , 0.9616791 , ..., 1.4066663 ,
          2.5305195 , 1.2188218 ],
         ...,
         [1.3998634 , 1.49087

In [13]:
import hashlib

def hash_file(fpath, algorithm='sha256', chunk_size=65535):
    """[Copyright Keras] Calculates a file sha256 or md5 hash.

    # Example
    ```python
        >>> from keras.utils.data_utils import _hash_file
        >>> _hash_file('/path/to/file.zip')
        'e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855'
    ```

    Arguments:

    - *fpath*: path to the file being validated
    - *algorithm*: hash algorithm, one of 'auto', 'sha256', or 'md5'.
           The default 'auto' detects the hash algorithm in use.
    - *chunk_size*: Bytes to read at a time, important for large files.

    Returns:
    The file hash
    """
    if (algorithm == 'sha256') or (algorithm == 'auto' and len(hash) == 64):
        hasher = hashlib.sha256()
    else:
        hasher = hashlib.md5()

    with open(fpath, 'rb') as fpath_file:
        for chunk in iter(lambda: fpath_file.read(chunk_size), b''):
            hasher.update(chunk)

    return hasher.hexdigest()



In [14]:
hash_file('resnet50_tensorpack_converted.h5', algorithm='md5')

'6bf74df9abc3c2e484e654112619530f'